<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-22 15:41:31
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.94 L
Current:  1.27 C
-------------------
Today PnL: -46.37 K (-0.36%)
Current PnL: -16.92 L (-12.0%)
CY Booked + Current PnL: -13.33 L (-9.45%)
-------------------
Total profit:  3.93 L
Total loss:  -20.85 L
-------------------
Total Booked + Current PnL: 13.65 L (11.97%)
Total Booked PnL: 30.57 L (26.81%)
Curr Year Booked PnL: 3.59 L (2.82%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 79.25 L (62.23%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.41 C (110.35%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 7.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
6     ANGELONE  2328.67  3033.00    15.94       H-SC  92.71  104890.0   
70     SBILIFE  1496.49  1924.99    -0.48       H-LC  71.88  147781.0   
4   ADANIPORTS  1103.69  1583.00    -2.94       M-LC  83.33   97369.0   
57  PIDILITIND  2504.06  3576.00   -18.89       X-LC  32.29   89835.0   
63    RELIANCE  1288.53  1526.00     3.47       X-LC  76.04  222685.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
6       21058.0   4300.0        1.65         25.12   4.10  30.25    149.0   
70      22076.0  13921.0       -0.26         17.56   9.42  28.63    159.0   
4       19007.0  15024.0       -0.95         24.26  15.43  43.43     48.0   
57      14713.0  17446.0        0.28         19.59  19.42  42.81    121.0   
63      19097.0  18416.0       -1.39          9.38   8.27  18.43    161.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
6      4.90        0.84    49.89     X40N      NTT     FINANCE  
70     1.59        1.19    31.65       AR      ATH   INSURANCE  
4      1.27        0.78    30.23     XY24      NTT        MISC  
57     0.84        0.72    12.66      X40      BTT   CHEMICALS  
63     1.04        1.79    21.30     XY25      NTT  REFINERIES

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
21   COFFEEDAY    59.14    80.0   -55.73       L-SC  98.96   66835.0   
67  SAMMAANCAP   170.35   326.0  -162.93       M-SC  45.83   75642.0   
14  BERGEPAINT   558.54   680.0   -16.37       H-MC  23.96  179549.0   
56      PFIZER  4236.33  6318.3     2.30       H-SC  97.92  159321.0   
75         SIS   477.00   528.0  2132.23       M-SC  61.46   60426.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
21     -46714.0   86765.0        4.44        -41.14  129.82  35.27    181.0   
67     -26568.0  119961.0        4.20        -25.99  158.59  91.37     54.0   
14       2492.0   36018.0        3.24          1.41   20.06  21.75    183.0   
56      27995.0   36548.0        3.22         21.32   22.94  49.15     94.0   
75     -20664.0   29331.0        3.01        -25.48   48.54  10.69    164.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
21    -0.54        0.54    63.58       XR      NTT   HOTELS  
67    -0.22        0.61    22.79     XY25      NTT  FINANCE  
14     0.07        1.44    28.44     XY24      NTT   PAINTS  
56     0.77        1.28    31.32      X40      ATH   PHARMA  
75    -0.70        0.49    22.46     X40N      NTT     MISC

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
22      COLPAL  2837.00  3808.06     1.65       X-MC   10.42  124355.0   
90  VALIANTORG   512.64   838.00  -335.18       H-SC   64.58   99905.0   
59       QUESS   711.66   986.00      NaN        NaN    5.21   72642.0   
68       SATIN   199.77   274.00   -19.84       H-SC   34.38  158991.0   
92      VIPIND   488.80   718.00  -848.59       H-SC  100.00   71321.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
22     -17495.0   66045.0       -6.48        -12.33   53.11  34.23     21.0   
90     -33381.0  117978.0       -4.79        -25.04  118.09  63.47    145.0   
59     -83212.0  143294.0       -2.36        -53.39  197.26  38.55     12.0   
68     -36983.0  109799.0       -2.30        -18.87   69.06  37.16    106.0   
92     -23995.0   68689.0       -2.08        -25.17   96.31  46.89    173.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
22    -0.26        1.00     5.81     XY25      ATH       FMCG  
90    -0.28        0.80    67.47       XR      NTT  CHEMICALS  
59    -0.58        0.58    11.23     XY24      NTT       MISC  
68    -0.34        1.28    18.73     XY24      NTT    FINANCE  
92    -0.35        0.57    40.67     X40N      NTT       MISC

In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
39   INDIAMART  2311.97  4911.36   -51.28       H-SC  58.33  105279.0   
83         TCS  3928.29  4998.00   -16.80       X-LC  59.38  187758.0   
82  TATAMOTORS   776.02  1065.00   -52.64       M-LC  90.62  190124.0   
10         AWL   342.88   485.00   -69.83       H-MC   3.12  163512.0   
42        INFY  1432.81  2275.00   -13.50       X-LC  75.00  215102.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
39       3552.0  110817.0        0.04          3.49  105.26  112.43      1.0   
83     -24370.0   82125.0       -1.38        -11.49   43.74   27.23      3.0   
82     -15521.0   92096.0       -1.29         -7.55   48.44   37.24      4.0   
10     -54217.0  144463.0       -1.06        -24.90   88.35   41.45      4.0   
42      15941.0  101119.0       -1.34          8.00   47.01   58.78      6.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
39     0.03        0.85    24.25       AR      ATH     MISC  
83    -0.30        1.51     7.60      X40      BTT       IT  
82    -0.17        1.53    23.75     XY24      NTT     AUTO  
10    -0.38        1.31     9.23     XY24      NTT     FMCG  
42     0.16        1.73    12.30      X40      BTT       IT

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
6     ANGELONE  2328.67  3033.0    15.94       H-SC  92.71  104890.0   
56      PFIZER  4236.33  6318.3     2.30       H-SC  97.92  159321.0   
4   ADANIPORTS  1103.69  1583.0    -2.94       M-LC  83.33   97369.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
6       21058.0   4300.0        1.65         25.12   4.10  30.25    149.0   
56      27995.0  36548.0        3.22         21.32  22.94  49.15     94.0   
4       19007.0  15024.0       -0.95         24.26  15.43  43.43     48.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
6      4.90        0.84    49.89     X40N      NTT  FINANCE  
56     0.77        1.28    31.32      X40      ATH   PHARMA  
4      1.27        0.78    30.23     XY24      NTT     MISC

In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost       FTT  Dev%_PE Conviction    RSP   Current  \
52    LUXIND  1448.19   2442.00    43.68       M-SC  72.92  148903.0   
6   ANGELONE  2328.67   3033.00    15.94       H-SC  92.71  104890.0   
58   POLYCAB  5005.95   7541.35     6.49       H-MC  67.71  101116.0   
56    PFIZER  4236.33   6318.30     2.30       H-SC  97.92  159321.0   
29  GILLETTE  8109.44  10330.69     0.60       H-SC  69.79  148334.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
52      11325.0  83088.0        0.11          8.23  55.80  68.62     66.0   
6       21058.0   4300.0        1.65         25.12   4.10  30.25    149.0   
58      16015.0  27089.0       -1.77         18.82  26.79  50.65     56.0   
56      27995.0  36548.0        3.22         21.32  22.94  49.15     94.0   
29      10474.0  27293.0       -1.38          7.60  18.40  27.39     64.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
52     0.14        1.20    29.09     X40N      NTT  TEXTILES  
6      4.90        0.84    49.89     X40N      NTT   FINANCE  
58     0.59        0.81    27.11     X40N      ATH    CABLES  
56     0.77        1.28    31.32      X40      ATH    PHARMA  
29     0.38        1.19    28.56      X40      ATH      FMCG

In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
18        CAMS  3643.00  5250.99    -3.33       H-SC  18.75  108713.0   
12   BANKINDIA   113.49   190.00   -31.87       M-MC  26.04  132520.0   
39   INDIAMART  2311.97  4911.36   -51.28       H-SC  58.33  105279.0   
95       WIPRO   243.46   420.00   -11.07       M-LC  60.42  152483.0   
34  HEROMOTOCO  4311.81  6039.03    -1.90       H-MC  84.38  149670.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
18       6709.0   38310.0       -0.13          6.58   35.24   44.14     31.0   
12       3368.0   83700.0       -1.28          2.61   63.16   67.42     57.0   
39       3552.0  110817.0        0.04          3.49  105.26  112.43      1.0   
95       1538.0  107912.0       -1.98          1.02   70.77   72.51     49.0   
34      -1243.0   61694.0        0.29         -0.82   41.22   40.06     27.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
18     0.18        0.87    23.89       XR      ATH     MISC  
12     0.04        1.07    28.89       XR      NTT    BANKS  
39     0.03        0.85    24.25       AR      ATH     MISC  
95     0.01        1.23    14.81       XR      NTT       IT  
34    -0.02        1.20    22.42       AR      ATH     AUTO

In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
21   COFFEEDAY    59.14    80.0   -55.73       L-SC  98.96   66835.0   
53      MASFIN   326.60   399.5   -23.67       H-SC  52.08   86625.0   
4   ADANIPORTS  1103.69  1583.0    -2.94       M-LC  83.33   97369.0   
90  VALIANTORG   512.64   838.0  -335.18       H-SC  64.58   99905.0   
6     ANGELONE  2328.67  3033.0    15.94       H-SC  92.71  104890.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
21     -46714.0   86765.0        4.44        -41.14  129.82  35.27    181.0   
53     -11355.0   33221.0       -0.16        -11.59   38.35  22.32    102.0   
4       19007.0   15024.0       -0.95         24.26   15.43  43.43     48.0   
90     -33381.0  117978.0       -4.79        -25.04  118.09  63.47    145.0   
6       21058.0    4300.0        1.65         25.12    4.10  30.25    149.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
21    -0.54        0.54    63.58       XR      NTT     HOTELS  
53    -0.34        0.70    25.64       XR      ATH    FINANCE  
4      1.27        0.78    30.23     XY24      NTT       MISC  
90    -0.28        0.80    67.47       XR      NTT  CHEMICALS  
6      4.90        0.84    49.89     X40N      NTT    FINANCE

In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
92     VIPIND   488.80   718.00  -848.59       H-SC  100.00   71321.0   
21  COFFEEDAY    59.14    80.00   -55.73       L-SC   98.96   66835.0   
56     PFIZER  4236.33  6318.30     2.30       H-SC   97.92  159321.0   
76   SONACOMS   578.05  1006.00   -28.54       M-MC   96.88   92409.0   
51       LTIM  5664.39  7262.39    -5.98       X-LC   95.83  150696.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
92     -23995.0  68689.0       -2.08        -25.17   96.31  46.89    173.0   
21     -46714.0  86765.0        4.44        -41.14  129.82  35.27    181.0   
56      27995.0  36548.0        3.22         21.32   22.94  49.15     94.0   
76      -8750.0  83639.0        0.57         -8.65   90.51  74.03     43.0   
51     -19236.0  67180.0       -0.21        -11.32   44.58  28.21     32.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
92    -0.35        0.57    40.67     X40N      NTT     MISC  
21    -0.54        0.54    63.58       XR      NTT   HOTELS  
56     0.77        1.28    31.32      X40      ATH   PHARMA  
76    -0.10        0.74    29.87       AR      BTT     AUTO  
51    -0.29        1.21    23.81     X200      ATH       IT

In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       17.27
1     25       38.45
2     50       67.69

In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.25
X40N    14.39
XY25    13.95
X40     13.52
AR      11.45
XR      10.08
X200     1.21
SR       1.13
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.42
M-SC    18.02
X-LC    16.18
H-MC     8.48
M-LC     7.67
H-LC     6.05
M-MC     5.67
X-MC     5.54
X-SC     2.88
L-SC     2.37
L-LC     1.19
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                11.59         -2.84   35.56
BANKS               11.06         -9.45   63.14
IT                   9.18        -21.24   88.88
FINANCE              6.94        -20.50   76.04
MISC                 6.49        -31.20   87.62
INSURANCE            6.26          0.78   29.13
PAINTS               5.61        -17.99   52.90
AUTO                 4.38        -10.91   59.26
ELECTRICAL           4.20         -6.21   60.33
HEALTHCARE           3.61         -3.68   37.15
CHEMICALS            3.39        -11.67   88.62
DURABLES             3.18         -0.45   69.95
FOOTWEAR             2.53        -30.47   73.58
AC                   1.89        -13.34   73.81
TEXTILES             1.85          0.11   52.35
TRAVEL               1.84        -38.95   99.08
REFINERIES           1.79          8.58    8.27
STEEL                1.74         -3.90   81.33
CERAMICS             1.70        -43.57  104.19
REALTY               1.40         11.62   32.99
JEWELLERY            1.29        -80.99  138.22
PHARMA               1.28         17.57   22.94
ENGINEERING          1.27        -21.47   71.37
MINING               1.19         -1.96   28.41
CEMENT               1.14        -32.65  100.89
ENTERTAINMENT        0.94        -40.46  111.08
POWER                0.89        -27.41   78.23
CABLES               0.81         15.84   26.79
HOTELS               0.54        -69.89  129.82

In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2986472.0     27
X40N      1203606.0     18
AR        1167760.0     13
XR        1095278.0     14
X40        721423.0     11
XY25       479090.0     10
SR         204207.0      2
X200        67180.0      1

In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2310777.0     22
M-SC        1997439.0     21
X-LC         728590.0     12
M-MC         563096.0      5
H-MC         467735.0      8
X-SC         350684.0      4
M-LC         321906.0      6
X-MC         315974.0      5
L-SC         303097.0      4
H-LC         218659.0      5
L-LC          41926.0      1

In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      918575.0      6
M-SC       XY24      759598.0      6
H-SC       AR        536016.0      3
           X40N      512062.0      7
M-SC       X40N      448462.0      6
M-MC       XY24      395757.0      3
X-LC       X40       392585.0      5
M-SC       XR        295581.0      4
H-SC       XR        280283.0      4
M-SC       X40       196678.0      2
M-LC       XY24      193393.0      4
H-LC       AR        187894.0      4
H-MC       XY24      180481.0      2
L-SC       XR        151723.0      2
X-MC       XY24      146447.0      2
X-LC       AR        141330.0      2
           XY25      127495.0      4
M-SC       XY25      119961.0      1
X-SC       SR        119949.0      1
           XR        118224.0      1
           X40N      112511.0      2
M-LC       XR        107912.0      1
X-MC       X40N      103482.0      2
M-SC       AR         92901.0      1
L-SC       XY24       87088.0      1
M-SC       SR         84258.0      1
M-MC       XR         83700.0      1
           AR         83639.0      1
H-MC       XY25       72297.0      1
           X40        68319.0      2
X-LC       X200       67180.0      1
X-MC       XY25       66045.0      1
L-SC       AR         64286.0      1
H-SC       X40        63841.0      2
H-MC       AR         61694.0      1
           XR         57855.0      1
L-LC       XY25       41926.0      1
H-LC       XY25       30765.0      1
H-MC       X40N       27089.0      1
M-LC       XY25       20601.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
